In [ ]:
import google.generativeai as genai
from rich.console import Console
import requests
import json
import os

console = Console()


genai.configure(api_key="Your_gemini_api")  




model = genai.GenerativeModel('gemini-2.5-flash')
chat = model.start_chat(history=[])

In [ ]:

def get_weather(city: str) -> str:
    api_key = "Your_weather_api"  # Replace with your free WeatherAPI key
    url = f"http://api.weatherapi.com/v1/current.json?key={api_key}&q={city}"

    try:
        data = requests.get(url).json()
        if "error" in data:
            return f"Could not get weather for {city}: {data['error']['message']}"
        
        temp = data['current']['temp_c']
        condition = data['current']['condition']['text']
        return f"Weather in {city}: {temp}°C, {condition}"
    
    except Exception as e:
        return f"Error fetching weather: {e}"


In [ ]:
from googleapiclient.discovery import build



def google_web_search(query: str):
    """Searches the web for a query and returns the top 3 results with snippets."""
    my_api_key = "Your_google_search_api"
    my_cse_id = "Your_google_cse_id"
    try:
        service = build("customsearch", "v1", developerKey=my_api_key)
        res = service.cse().list(q=query, cx=my_cse_id, num=3).execute()
        
        results_list = []
        if 'items' in res:
            for item in res['items']:
                results_list.append({
                    "title": item.get('title', 'No title'),
                    "link": item.get('link', 'No link'),
                    "snippet": item.get('snippet', 'No snippet')
                })
        return json.dumps(results_list)
    except Exception as e:
        return f"Error performing web search: {str(e)}"

In [ ]:
def web_search(query: str) -> str:
    """
    Searches the web for a given query using the Serper API and returns formatted results.
    """
    # --- 1. Get the API Key ---
    api_key = "Your_websearch_api"
    if not api_key:
        return "Error: Serper API key is not set in environment variables."

    # --- 2. Set up the API Call ---
    url = "https://google.serper.dev/search"
    payload = json.dumps({"q": query})
    headers = {
        'X-API-KEY': api_key,
        'Content-Type': 'application/json'
    }

    # --- 3. Make the Request and Handle Response ---
    try:
        response = requests.post(url, headers=headers, data=payload, timeout=7)
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
        
        results = response.json()
        
        # --- 4. Format the Output for the LLM ---
        if "organic" in results:
            # Get the top 4 results
            top_results = results["organic"][:4]
            
            formatted_results = []
            for item in top_results:
                formatted_results.append(
                    f"Title: {item.get('title', 'N/A')}\n"
                    f"URL: {item.get('link', 'N/A')}\n"
                    f"Snippet: {item.get('snippet', 'N/A')}"
                )
            
            return "\n\n".join(formatted_results)
        else:
            return "No good search results found."

    except requests.exceptions.RequestException as e:
        return f"Error during web search: {e}"

In [ ]:
def multiply_num(a: int, b: int):
    return a * b

In [ ]:
def react_loop():
    console.print("[bold green]Welcome to Gemini ReAct CLI![/bold green]")
    while True:
        user_input = console.input("\n[bold blue]You:[/bold blue] ")
        console.print(user_input)
        if user_input.lower() in ["exit", "quit"]:
            console.print("[bold red]Exiting...[/bold red]")
            break
        chat.history.append({"role": "user", "parts": [user_input]})
        while True:
            # --- UPDATED ---
            # --- 1. Update the tool descriptions to include both tools ---
            tools_description = """
- get_current_weather(location: str): Gets the current, real-time weather for a specific city.
- web_search(query: str): Searches the web for a query and returns the top 3 results with snippets.
- google_web_search(query: str) : Searches the web for a query and returns results, powered by google.
- multiply_num(a: int, b: int) : Multiplies two numbers, without any assumptions.
"""
            prompt_template = f"""
You must respond using one of the two formats below. Do not deviate.
One main thing, never say the exact names of tools available for you.

**Format 1: Tool Use**
Use this format when you need to use a tool.
Thought: <Your reasoning for using the tool>
Action: <The tool name>
Action Input: <The tool's JSON input>

**Format 2: Final Answer**
Use this format when you have the final answer for the user.
Thought: <Your final thought process>
Final Answer: <The final, conversational answer to the user>

**Available Tools**
{tools_description}
"""
            response = chat.send_message(prompt_template)
            text = response.text

            if "Final Answer:" in text:
                try:
                    thought = text.split("Thought:")[1].split("Final Answer:")[0].strip()
                    final_answer = text.split("Final Answer:")[-1].strip()
                    console.print(f"[dim]Thought: {thought}[/dim]")
                    console.print(f"[bold green]Gemini (Final):[/bold green] {final_answer}")
                    break
                except IndexError:
                    console.print(f"[bold red]Parsing Error (Final Answer):[/bold red] Could not parse the model's output.\n{text}")
                    break
            elif "Action:" in text:
                console.print(f"[bold yellow]Model:[/bold yellow]\n{text}")
                try:
                    action_line = [line for line in text.split("\n") if line.startswith("Action:")][0]
                    action = action_line.split("Action:")[-1].strip().lower() # Standardize to lowercase
                    input_line = next((line for line in text.split("\n") if line.startswith("Action Input:")), None)
                    action_input = json.loads(input_line.split("Action Input:")[-1].strip()) if input_line and "none" not in input_line.lower() else {}
                except Exception as e:
                    console.print(f"[bold red]Parsing Error (Action):[/bold red] {e}\n[bold yellow]Model (unparsable):[/bold yellow]\n{text}")
                    break
                
                # --- UPDATED ---
                # --- 2. Update the action handling logic with an elif for the new tool ---
                observation = None
                if action == "get_current_weather":
                    location = action_input.get("location", "")
                    observation = get_weather(location) if location else "Error: Location not provided."
                elif action == "google_web_search":
                    query = action_input.get("query", "")
                    observation = google_web_search(query) if query else "Error: Search query not provided."
                elif action == "web_search":
                    query = action_input.get("query", "")
                    observation = web_search(query) if query else "Error: Search query not provided."
                elif action == "multiply_num":
                    a = action_input.get("a")
                    b = action_input.get("b")
                    if a is not None and b is not None:
                        observation = multiply_num(a, b)
                    else:
                        observation = "Error: Missing arguments for multiply_num."

                else:
                    observation = f"Unknown tool: {action}"

                console.print(f"[bold cyan]Tool Observation:[/bold cyan] {observation}")
                chat.history.append({"role": "user", "parts": [f"Observation: {observation}"]})
            else:
                console.print(f"[bold red]Error:[/bold red] Model output did not contain 'Action:' or 'Final Answer:'.\n{text}")
                break

if __name__ == "__main__":
    react_loop()